<style>
div.blue{
    background-color:#e6f0ff; 
    border-radius: 5px; 
    padding: 20px;}
</style> 

<style>
div.warn {    
    background-color: #fcf2f2;
    border-color: #dFb5b4;
    border-left: 5px solid #dfb5b4;
    padding: 0.5em;
    }
 </style>
    
<h1 style="text-align: center; color: purple;" markdown="1">Econ 320 Python Lab Simple Regression </h1>
<h2 style="text-align: center; color: purple;" markdown="1">Handout 5</h2>

## Simple Regression Part 2

In [79]:
# Import your packages 
import wooldridge as woo
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Now import the package to run regressions
import statsmodels.formula.api as smf

In [80]:
# Import data wage1 form Wooldridge
wage1 = woo.dataWoo('wage1')
# Read CSV file into a DataFrame this data comes from the web 
# http://672258.youcanlearnit.net/college.csv
collegedata = pd.read_csv('http://672258.youcanlearnit.net/college.csv')
wage1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 24 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   wage      526 non-null    float64
 1   educ      526 non-null    int64  
 2   exper     526 non-null    int64  
 3   tenure    526 non-null    int64  
 4   nonwhite  526 non-null    int64  
 5   female    526 non-null    int64  
 6   married   526 non-null    int64  
 7   numdep    526 non-null    int64  
 8   smsa      526 non-null    int64  
 9   northcen  526 non-null    int64  
 10  south     526 non-null    int64  
 11  west      526 non-null    int64  
 12  construc  526 non-null    int64  
 13  ndurman   526 non-null    int64  
 14  trcommpu  526 non-null    int64  
 15  trade     526 non-null    int64  
 16  services  526 non-null    int64  
 17  profserv  526 non-null    int64  
 18  profocc   526 non-null    int64  
 19  clerocc   526 non-null    int64  
 20  servocc   526 non-null    int64 

**Run a regression for the wages**

Based on the variables in our data set, we will run two models, the first model is a simple regression of education on wages then a multiple regression object adding experience and tenure. 

model 1 $$ log(wage) = \beta_0 + \beta_1*education + u $$
model 2 $$ log(wage) = \beta_0 + \beta_1*education + \beta_2*experience + \beta_3*tenure + u $$

In [81]:
# Inspect the data with a correlation table 
wage1[["wage","educ","exper","tenure"]].corr()

,wage,educ,exper,tenure
wage,1.000000,0.405903,0.112903,0.346890
educ,0.405903,1.000000,-0.299542,-0.056173
exper,0.112903,-0.299542,1.000000,0.499291
tenure,0.346890,-0.056173,0.499291,1.000000


In [82]:
# Name create the regression objects from model1 and model2  
model1 = smf.ols(formula= "lwage ~ educ",data = wage1).fit()
model2 =smf.ols(formula="lwage ~ educ + exper + tenure",data = wage1).fit()

In [83]:
# Use the method .params to obtain the beta parameters, create a table for comparing both models 
model1.params
model2.params
table = pd.DataFrame({"Betas1":round(model1.params,3),"Beta2":round(model2.params,3)})
table


,Betas1,Beta2
Intercept,0.584,0.284
educ,0.083,0.092
exper,NaN,0.004
tenure,NaN,0.022


In [84]:
# show R^2 for both models 

print("Model 1 R^2:", round(model1.rsquared,3))
print("Model 2 R^2:", round(model2.rsquared,3))

Model 1 R^2: 0.186
Model 2 R^2: 0.316


In [85]:
# Now all this information can be seen on a table using the method .summary(). Do it for model 1 
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.186
Model:                            OLS   Adj. R-squared:                  0.184
Method:                 Least Squares   F-statistic:                     119.6
Date:                Fri, 14 Feb 2025   Prob (F-statistic):           3.27e-25
Time:                        13:55:58   Log-Likelihood:                -359.38
No. Observations:                 526   AIC:                             722.8
Df Residuals:                     524   BIC:                             731.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5838      0.097      5.998      0.000       0.393       0.775
educ           0.0827      0.008     10.935      0.000       0.068       0.098
==============================================================================
Omnibus:                       11.804   Durbin-Watson:                   1.801
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               13.811
Skew:                           0.268   Prob(JB):                      0.00100
Kurtosis:                       3.586   Cond. No.                         60.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [86]:
# Extract the number of observations 
print("Model1 number of observations:", model1.nobs)

Model1 number of observations: 526.0


Now, we know that the mean of $\hat{y}$ = mean of $y$

Let's prove this with our model1, estimating al the values of $\hat{y}$ and then calculating both means 


In [87]:

# save the mean of yhat using .fittedvalues
wage1["lwage_hat1"]=model1.fittedvalues
# pirnt both means 
print("mean of yhat:", round(wage1.lwage_hat1.mean(),3))
print("mean of y:", round(wage1.lwage.mean(),3))

# show the first 10 observations for y 
wage1[["lwage_hat1", "lwage"]].head()

mean of yhat: 1.623
mean of y: 1.623


,lwage_hat1,lwage
0,1.493961,1.131402
1,1.576705,1.175573
2,1.493961,1.098612
3,1.245728,1.791759
4,1.576705,1.667707


Now show that the sum of residuals  is equal to zero 
$$ \sum{e_i}=0$$

Show this for model 1, To make this easier, round(  , 3) and take the absolute value as the sum is $\approx  0$

In [88]:
# the method .resid allows you to obtain the residuals of the model. 
round(model1.resid.mean(),3)

-0.0

**College Data Exercise**

Now take a look at the college data that you are familiar with from Econ 220. Look at the variables and think about a model that you can estimate with this data, start with just a simple regression. What will be your Y and your X and why, run the regression and interpret the results. 

In [89]:
collegedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1269 entries, 0 to 1268
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1269 non-null   int64  
 1   name                1269 non-null   object 
 2   city                1269 non-null   object 
 3   state               1269 non-null   object 
 4   region              1269 non-null   object 
 5   highest_degree      1269 non-null   object 
 6   control             1269 non-null   object 
 7   gender              1269 non-null   object 
 8   admission_rate      1269 non-null   float64
 9   sat_avg             1269 non-null   int64  
 10  undergrads          1269 non-null   int64  
 11  tuition             1269 non-null   int64  
 12  faculty_salary_avg  1269 non-null   int64  
 13  loan_default_rate   1267 non-null   float64
 14  median_debt         1269 non-null   float64
 15  lon                 1269 non-null   float64
 16  lat   

$$ Y = \beta_0 + \beta_1* X + u $$

In [90]:
model3 = smf.ols(formula = "admission_rate ~ I(sat_avg/100) + control + np.log(tuition) ",data = collegedata).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         admission_rate   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.142
Method:                 Least Squares   F-statistic:                     71.04
Date:                Fri, 14 Feb 2025   Prob (F-statistic):           1.84e-42
Time:                        13:55:59   Log-Likelihood:                 391.60
No. Observations:                1269   AIC:                            -775.2
Df Residuals:                    1265   BIC:                            -754.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.7453      0.155      4.819      0.000       0.442       1.049
control[T.Public]     0.0907      0.022      4.119      0.000       0.048       0.134
I(sat_avg / 100)     -0.0577      0.005    -12.778      0.000      -0.067      -0.049
np.log(tuition)       0.0492      0.017      2.867      0.004       0.016       0.083
==============================================================================
Omnibus:                       23.329   Durbin-Watson:                   1.523
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.519
Skew:                          -0.312   Prob(JB):                     7.82e-06
Kurtosis:                       2.766   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [91]:
!jupyter nbconvert --to html H4_F24_320Lab_Simple_Multi.ipynb

This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Execute the notebook prior to export.
    Equivalent to: [--ExecutePr

[NbConvertApp] WARNING | pattern 'H4_F24_320Lab_Simple_Multi.ipynb' matched no files
